- autovacuum 데몬으로 진공 처리가 자동화되어 PostgreSQL 작업이 쉬워짐

- autovacuum 데몬은 여러 autovacuum_worker 프로세스를 주기적으로 호출.
  - 기본적으로 1분마다 깨어나고( autovacuum_naptime 으로 정의됨 ) 3개의 작업자( autovacuum_max_works 로 정의됨 )를 호출.

- autovacuum 데몬이 호출한 Autovacuum 작업자는 데이터베이스 활동에 최소한의 영향을 미치면서 점진적으로 각 테이블에 대해 동시에 Vacuum 처리를 수행.



# **Conditions for autovacuum to run**


- 다음 조건 중 하나라도 충족되면 대상 테이블에 대해 Autovacuum 프로세스가 실행됨.

1. 현재 txid는 다음 표현식 앞에 옴.

<br>

> relfrozenxid + autovacuum_freeze_max_age

<br>

- 여기서 relfrozenxid는 pg_class에 정의된 대상 테이블의 relfrozenxid 값이고, autovacuum_freeze_max_age(기본값은 200,000,000)는 구성 매개변수.
- 이 조건이 만족되면 대상 테이블에 대해 Autovacuum 프로세스가 실행되어 동결 처리가 수행됨.

2. 데드 튜플 수가 다음 표현식보다 큼.

<br>

> autovacuum_vacuum_threshold + autovacuum_vacuum_scale_factor × reltuples

- 여기서 autovacuum_vacuum_threshold (기본값은 50) 및
autovacuum_vacuum_scale_factor (기본값은 0.2)는 구성 매개변수.
- reltuples 대상 테이블의 튜플 수

- 예를 들어 대상 테이블에 10,000개의 튜플과 2,100개의 데드 튜플이 있는 경우 대상 테이블에 대해 autovacuum 프로세스가 실행됨.

> 2100 > 50 + 0.2 × 10000.

3. 대상 테이블에 삽입된 튜플 수가 다음 식보다 큼.

> autovacuum_vacuum_insert_threshold + autovacuum_vacuum_insert_scale_factor × reltuples

- 여기서 autovacuum_vacuum_insert_threshold (기본값은 1000) 및 autovacuum_vacuum_insert_scale_factor (기본값은 0.2)는 구성 매개변수.
- reltuples  대상 테이블의 튜플 수

- 예를 들어 대상 테이블에 10,000개의 튜플이 있고 3,010개의 삽입된 튜플이 있는 경우 Autovacuum 프로세스가 대상 테이블에 대해 실행됨.

> 3010 > 1000 + 0.2 × 10000.

- 이 조건은 버전 13부터 추가되었음.

- 또한 대상 테이블에 대해 다음 조건이 만족되면 Autovacuum 프로세스도 분석 처리를 수행.

- 여기서 mod_since_analyze는 이전 분석 처리 이후 수정된 튜플(INSERT, DELETE 또는 UPDATE에 의해) 수이고, autovacuum_analyze_threshold(기본값은 50) 및 autovacuum_analyze_scale_factor(기본값은 0.1)는 구성 매개변수.

- reltuples  대상 테이블의 튜플 수.

- 예를 들어 이전 분석 처리 이후 대상 테이블에 10,000개의 튜플과 1,100개의 수정된 튜플이 있는 경우 autovacuum 프로세스가 실행됨.

> 1100 > 50 + 0.1 × 10000.

**info**

- Relation_needs_vacanalyze () 함수는 대상 테이블을 정리하거나 분석해야 하는지 여부를 결정.

# **Maintenance tips**

- 테이블 팽창은 PostgreSQL 관리에 있어 어려운 일 중 하나
- 여러 가지 요인으로 인해 이 문제가 발생할 수 있으며 Autovacuum이 그 중 하나.

- Autovacuum은 데드 튜플 수가 1,000개 관계의 경우 250개, 100,000개 관계의 경우 20,050개, 100,000,000개 관계의 경우 20,000,050개보다 클 때 실행됨.
- 이러한 예를 보면 테이블에 튜플이 많을수록 autovacuum 실행 빈도가 줄어든다는 것이 분명.

- 잘 알려진 팁은 autovacuum_vacuum_scale_factor 값을 줄이는 것.
  - 실제로 기본값(0.2)은 큰 테이블에 비해 너무 큼.

- PostgreSQL은 ALTER TABLE 명령을 사용하여 각 테이블에 적절한 autovacuum_vacuum_scale_factor를 설정할 수 있음
- 예를 들어 pgbench_accounts 테이블에 대해 autovacuum_vacuum_scale_factor의 새 값을 설정하는 방법을 보여줌.



```
postgres=# ALTER TABLE pgbench_accounts SET (autovacuum_vacuum_scale_factor = 0.05);
ALTER TABLE
```

- 튜플 수에 관계없이 대상 테이블에 대해 Autovacuum을 실행해야 하는 경우 이를 수행할 수도 있음.

- 예를 들어 데드 튜플 수가 10,000개에 도달할 때마다 Autovacuum 처리가 필요하다고 가정.
- 이 경우 테이블에 대해 다음 스토리지 매개변수를 설정하면 Autovacuum 프로세스는 10,000에 도달할 때마다 진공 처리를 수행.

```
postgres=# ALTER TABLE pgbench_accounts SET (autovacuum_vacuum_threshold = 10000);
ALTER TABLE
postgres=# ALTER TABLE pgbench_accounts SET (autovacuum_vacuum_scale_factor = 0.0);
ALTER TABLE
```